In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations

import numpy as np
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

import os
os.environ['PYTHONHASHSEED'] = '42'

keras.utils.set_random_seed(42)
# tf.config.experimental.enable_op_determinism()


In [2]:
height,width=32,32
batch_size=64

In [3]:
from src.augmentations.CustomAugmentations import CustomAugmentationsTF

# Define the augmentation probabilities
p_dict = {
  "flip": 0.5,
  # "transpose": 0.5,
  "gauss_noise": 0.5,
  # "brightness_contrast": 0.5,
  # "hue_saturation_value": 0.5,
}

p_dicts = []
# for aug_type in p_dict.keys():
#     p_d = {aug_type: .5}
#     p_dicts.append(p_d)

# p_dict = {
#   "flip": 0.5,
#   "transpose": 0.5,
#   "gauss_noise": 0.5,
#   "brightness_contrast": 0.5,
#   "hue_saturation_value": 0.5 ,
# }
p_dicts.append(p_dict)

p_dicts


[{'flip': 0.5, 'gauss_noise': 0.5}]

In [4]:
import matplotlib.pyplot as plt

augmented = original_dataset.map(lambda image, label: (custom_aug.augment(image), label))



In [5]:
from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from src.kerasCNN import *
import pickle


EPOCHS = 50

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )
        dnn_model = Sequential()
        imported_model= tf.keras.applications.ResNet50 (include_top=False,
        input_shape=(height,width,3),
        pooling='max',classes=10,
        weights='imagenet')

        dnn_model.add(imported_model)
        dnn_model.add(Flatten())
        dnn_model.add(Dense(2048, activation='relu'))
        dnn_model.add(Dense(1024, activation='relu'))
        dnn_model.add(Dense(10, activation='softmax'))

        dnn_model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
       

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/ResNet50_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))


    with open(f"results/HISTORY_ResNet50_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_ResNet50_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_ResNet50_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_ResNet50_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)

Augmentation:   0%|          | 0/1 [00:00<?, ?it/s]

Augmentations: flip0.5_gauss_noise0.5


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch 1/50
1336/1336 [==============================] - 64s 46ms/step - loss: 1.6118 - accuracy: 0.4175 - val_loss: 2.5572 - val_accuracy: 0.2451
Epoch 2/50
1336/1336 [==============================] - 60s 45ms/step - loss: 1.3862 - accuracy: 0.5051 - val_loss: 1.3967 - val_accuracy: 0.5102
Epoch 3/50
1336/1336 [==============================] - 60s 45ms/step - loss: 1.3728 - accuracy: 0.5090 - val_loss: 1.3233 - val_accuracy: 0.5260
Epoch 4/50
1336/1336 [==============================] - 60s 45ms/step - loss: 1.1602 - accuracy: 0.5862 - val_loss: 1.4407 - val_accuracy: 0.5073
Epoch 5/50
1336/1336 [==============================] - 60s 45ms/step - loss: 1.1132 - accuracy: 0.6080 - val_loss: 1.3660 - val_accuracy: 0.5276
Epoch 6/50
1336/1336 [==============================] - 60s 45ms/step - loss: 1.1203 - accuracy: 0.6056 - val_loss: 1.4561 - val_accuracy: 0.5062
Epoch 7/50
1336/1336 [==============================] - 60s 45ms/step - loss: 1.1551 - accuracy: 0.5920 - val_loss: 1.3741 -

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 64s 46ms/step - loss: 1.4613 - accuracy: 0.4853 - val_loss: 1.5064 - val_accuracy: 0.4882
Epoch 2/50
1336/1336 [==============================] - 60s 45ms/step - loss: 1.1748 - accuracy: 0.5900 - val_loss: 1.6902 - val_accuracy: 0.4409
Epoch 3/50
1336/1336 [==============================] - 58s 43ms/step - loss: 1.0624 - accuracy: 0.6308 - val_loss: 1.2061 - val_accuracy: 0.5958
Epoch 4/50
1336/1336 [==============================] - 55s 41ms/step - loss: 1.0260 - accuracy: 0.6430 - val_loss: 1.2581 - val_accuracy: 0.5802
Epoch 5/50
1336/1336 [==============================] - 55s 41ms/step - loss: 1.1537 - accuracy: 0.6016 - val_loss: 1.3268 - val_accuracy: 0.5462
Epoch 6/50
1336/1336 [==============================] - 56s 42ms/step - loss: 1.0710 - accuracy: 0.6255 - val_loss: 1.2142 - val_accuracy: 0.5800
Epoch 7/50
1336/1336 [==============================] - 55s 41ms/step - loss: 1.1116 - accuracy: 0.6104 - val_loss: 1.1371 -

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 1/50
1336/1336 [==============================] - 61s 43ms/step - loss: 1.5086 - accuracy: 0.4672 - val_loss: 1.5279 - val_accuracy: 0.4978
Epoch 2/50
1336/1336 [==============================] - 56s 42ms/step - loss: 1.3106 - accuracy: 0.5342 - val_loss: 1.6061 - val_accuracy: 0.4500
Epoch 3/50
1336/1336 [==============================] - 56s 42ms/step - loss: 1.1303 - accuracy: 0.6000 - val_loss: 1.9451 - val_accuracy: 0.4600
Epoch 4/50
1336/1336 [==============================] - 56s 42ms/step - loss: 1.1020 - accuracy: 0.6125 - val_loss: 1.3348 - val_accuracy: 0.5667
Epoch 5/50
1336/1336 [==============================] - 56s 42ms/step - loss: 1.0503 - accuracy: 0.6316 - val_loss: 1.4347 - val_accuracy: 0.5318
Epoch 6/50
1336/1336 [==============================] - 56s 42ms/step - loss: 0.9924 - accuracy: 0.6511 - val_loss: 1.3189 - val_accuracy: 0.5573
Epoch 7/50
1336/1336 [==============================] - 56s 42ms/step - loss: 0.8992 - accuracy: 0.6832 - val_loss: 1.1148 -